## Train neural network

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append("/content/drive/MyDrive/git_workspace/mlconservation_code/src")
    print ("Running on google drive")
except:
    print ("Not running on google drive")

import numpy as np
import tensorflow as tf

from data_generator import DataGenerator
from dynamical_system import XYModelSystem
from nn_models import XYModelNNLagrangian, LagrangianModel
from initializer import XYModelRandomInitializer, XYModelConstantInitializer
%load_ext autoreload
%autoreload 2

In [2]:
BATCH_SIZE = 16
EPOCHS = 200
STEPS_PER_EPOCH=100
dim = 4
sigma = 0.001
sample_full_phase_space = False

if sample_full_phase_space:
    initializer = XYModelRandomInitializer(dim)
    re_initialize = True
else:
    initializer = XYModelConstantInitializer(dim)
    re_initialize = False

dynamical_system = XYModelSystem(dim)
data_generator = DataGenerator(dynamical_system, 
                               initializer,
                               re_initialize=re_initialize,
                               sigma=sigma)
train_batches = data_generator.dataset.batch(BATCH_SIZE)
nn_lagrangian = XYModelNNLagrangian(dim,rotation_invariant=True,shift_invariant=True)
model = LagrangianModel(nn_lagrangian)

In [ ]:
learning_rate = 1.E-4
model.compile(
    loss="mse",
    metrics=[],
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
)
log_dir = './tb_logs/'
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

result = model.fit(train_batches,
                   epochs=EPOCHS,
                   steps_per_epoch=STEPS_PER_EPOCH,
                   callbacks=[tensorboard_cb])


In [ ]:
nn_lagrangian.save("../assests/saved_models/nn_lagrangian26Jul2022")